<a href="https://colab.research.google.com/github/mparker2103/PatternFlow/blob/topic-recognition/Improved_UNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage.io import imread_collection, imread

from google.colab import drive

In [2]:
depth = 64
n = 32 #image dim
batch_size = 250
epochs = 20
categories = 10

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Read single image from folders; this caches the folder to get Google Colab to iterate through files quicker
train = imread('/content/drive/MyDrive/Colab_Notebooks/Data/ISIC2018_Task1-2_Training_Input_x2/ISIC_0000000.jpg')
test = imread('/content/drive/MyDrive/Colab_Notebooks/Data/ISIC2018_Task1_Training_GroundTruth_x2/ISIC_0000000_segmentation.png')

In [7]:
# Load the data
dir_train =    "/content/drive/MyDrive/Colab_Notebooks/Data/ISIC2018_Task1-2_Training_Input_x2/*.jpg"
dir_test =     "/content/drive/MyDrive/Colab_Notebooks/Data/ISIC2018_Task1_Training_GroundTruth_x2/*.png"

train = imread_collection(dir_train)

In [ ]:
test = imread_collection(dir_test)

In [ ]:
# Check images loaded correctly
plt.imshow(train[0], cmap='gray')
plt.show()

In [ ]:
# Convert image collections to tensors and normalise
train = tf.convert_to_tensor(train)
test = tf.convert_to_tensor(test)

In [ ]:
train, validate = train_test_split(train, test_size=0.2, random_state=1234)

In [ ]:
train = tf.cast(train, tf.float32) / 255.
validate = tf.cast(validate, tf.float32) / 255.
test = tf.cast(test, tf.float32) / 255.

print(train.shape)
print(validate.shape)
print(test.shape)

In [ ]:
# create the network
inputs = Input(shape=(n,n,3))
net1 = Conv2D(depth, (3,3), padding='same', activation='relu')(inputs)
pool1 = MaxPooling2D(pool_size=(2,2))(net1)
# dropout1 = Dropout(rate=.1)(pool1)
net2 = Conv2D(depth*4, (3,3), padding='same', activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2,2), strides=(1,1))(net2)
dropout2 = Dropout(rate=.2)(pool2)
net3 = Conv2D(depth*8, (3,3), padding='same', activation='relu')(dropout2)
pool3 = MaxPooling2D(pool_size=(2,2))(net3)
# dropout3 = Dropout(rate=.3)(pool3)
net4 = Conv2D(depth*16, (3,3), padding='same', activation='relu')(pool3)
pool4 = MaxPooling2D(pool_size=(2,2), strides=(1,1))(net4)
dropout4 = Dropout(rate=.4)(pool4)
flat = Flatten()(dropout4)
net5 = Dense(depth*32, activation='relu')(flat)
output = Dense(categories, activation='softmax')(net5)

In [ ]:
# Map an input to its reconstruction to create model
model = Model(inputs, output)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_hist = model.fit(X_train, y_train_cat,
            epochs=epochs,
            batch_size=batch_size,
            shuffle=True,
            validation_data=(X_val, y_val_cat))